In [1]:
from keras.models import Model
from keras.layers import Input, Embedding, merge

import numpy as np
import pickle
import gensim

C:\Users\fatih.barmanbay\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\fatih.barmanbay\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
embedding_matrix = np.load("word2vec_data/embeddings.npy")
word_2_index = pickle.load(open('word2vec_data/word_2_index.pkl', 'rb'))
index_2_word = pickle.load(open('word2vec_data/index_2_word.pkl', 'rb'))
model = gensim.models.Word2Vec.load("word2vec_data/my_word2vec")
wv = model.wv

In [3]:
def cos_similarity(v1, v2):
    numerator = np.dot(v1, v2)
    denominator = np.sqrt(np.sum(np.power(v1, 2))) * np.sqrt(np.sum(np.power(v2, 2)))
    
    return numerator / denominator

In [4]:
def word_similarity(w1, w2):
    idx1 = word_2_index[w1]
    idx2 = word_2_index[w2]

    v1 = embedding_matrix[idx1]
    v2 = embedding_matrix[idx2]
    return cos_similarity(v1, v2)

In [5]:
word_similarity("erkek", "kadın")

0.5155166151247436

In [6]:
# input words - in this case we do sample by sample evaluations of the similarity
valid_word = Input((1,), dtype='int32')
other_word = Input((1,), dtype='int32')

# setup the embedding layer
embeddings = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                      weights=[embedding_matrix])

embedded_a = embeddings(valid_word)
embedded_b = embeddings(other_word)
similarity = merge([embedded_a, embedded_b], mode='cos', dot_axes=2)

C:\Users\fatih.barmanbay\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\fatih.barmanbay\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [7]:
# create the Keras model
k_model = Model(inputs=[valid_word, other_word], outputs=similarity)

In [8]:
def get_sim(valid_word_idx, vocab_size):
    sim = np.zeros((vocab_size,))
    in_arr1 = np.zeros((1,))
    in_arr2 = np.zeros((1,))
    in_arr1[0,] = valid_word_idx
    for i in range(vocab_size):
        in_arr2[0,] = i
        out = k_model.predict_on_batch([in_arr1, in_arr2])
        sim[i] = out
    return sim

In [9]:
valid_size = 15  # Random set of words to evaluate similarity on.
valid_window = 250  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [10]:
# now run the model and get the closest words to the valid examples
for i in range(valid_size):
    valid_word = index_2_word[valid_examples[i]]
    top_k = 8  # number of nearest neighbors
    sim = get_sim(valid_examples[i], len(wv.vocab))
    nearest = (-sim).argsort()[1:top_k + 1]
    log_str = "Nearest to '%s':" % valid_word
    for k in range(top_k):
        close_word = index_2_word[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)

Nearest to 'ki': nsanlar, nanılmaz, nşallah, nsanların, şin, kisi, sterse, şe,
Nearest to 'o': dediğiniz, oynadığım, onlar, bu, dediğim, öyle, istediğin, binnur,
Nearest to 'takım': takımın, takımımız, takımda, ekip, takımlar, takımımızın, takıma, takımı,
Nearest to 'şey': şeyi, birşey, şeyler, şeyimiz, şeyim, şeyin, şeye, şeyde,
Nearest to 'geldi': gelmişti, gelirken, gelmiştir, geliyor, geldik, gelebilir, geldiler, gelen,
Nearest to 'bunun': onun, bunların, düşeni, bunu, sözünün, denmesi, öğrenmesi, sorması,
Nearest to 'diye': şeklinde, diyerek, deyip, neyi, sözleriyle, bilmediğim, yoksa, dedim,
Nearest to 'çıktı': çıkmıştı, çıkarıldı, çıkıyor, atılmıştı, çıkmıştır, çıkan, çıkınca, çıkarken,
Nearest to 'hem': hemde, kulübüm, kültürle, rahatlatmak, stattaki, kalitesini, gelişmesine, başarısıyla,
Nearest to '12': 17, 22, 15, 14, 11, 7, 18, 27,
Nearest to 'beşiktaş': galatasaray, bursaspor, fenerbahçe, niang, trabzonspor, sivasspor, eskişehirspor, ktaş,
Nearest to 'neden': sebep, şahit,